In [38]:
import pandas as pd
file_path = "/home/deeptanshu/.cache/kagglehub/datasets/ealaxi/paysim1/versions/2/PS_20174392719_1491204439457_log.csv"
df = pd.read_csv(file_path)
print(df.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [20]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
None


In [21]:
print(df.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['amount_normalized'] = scaler.fit_transform(df[['amount']])

In [39]:
import joblib
df.drop(['nameOrig', 'nameDest'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['type'], drop_first=True)
df['errorBalanceOrig'] = df['newbalanceOrig'] + df['amount'] - df['oldbalanceOrg']
df['errorBalanceDest'] = df['oldbalanceDest'] + df['amount'] - df['newbalanceDest']
joblib.dump(list(df.columns), 'scaler_columns.pkl')


['scaler_columns.pkl']

In [24]:
df['timestamp'] = pd.date_range(start='2025-01-01', periods=len(df), freq='min')

In [25]:
import joblib
X = df.drop(['isFraud', 'isFlaggedFraud', 'timestamp'], axis=1, errors='ignore')
y = df['isFraud']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print("Logistic Regression Results:\n", classification_report(y_test, y_pred))

import joblib
joblib.dump(logreg, 'log_model.pkl')

Logistic Regression Results:
               precision    recall  f1-score   support

           0       1.00      0.95      0.97   1270881
           1       0.02      0.96      0.05      1643

    accuracy                           0.95   1272524
   macro avg       0.51      0.96      0.51   1272524
weighted avg       1.00      0.95      0.97   1272524



['log_model.pkl']

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Random Forest Results:\n", classification_report(y_test, y_pred))

import joblib
joblib.dump(rf, 'rf_model.pkl')


Random Forest Results:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       1.00      1.00      1.00      1643

    accuracy                           1.00   1272524
   macro avg       1.00      1.00      1.00   1272524
weighted avg       1.00      1.00      1.00   1272524



['rf_model.pkl']

In [29]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1270881       0]
 [      4    1639]]


In [30]:
from sklearn.ensemble import IsolationForest
import joblib

iso = IsolationForest(contamination=0.001, random_state=42)
iso.fit(X_scaled)

joblib.dump(iso, 'iso_model.pkl')


['iso_model.pkl']

In [31]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Train
iso = IsolationForest(contamination=0.005)
iso.fit(X_train)

# Predict
y_pred_iso = iso.predict(X_test)
y_pred_iso = [1 if p == -1 else 0 for p in y_pred_iso]

# Evaluate
print("Isolation Forest Report:")
print(confusion_matrix(y_test, y_pred_iso))
print(classification_report(y_test, y_pred_iso))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_iso))


Isolation Forest Report:
[[1264719    6162]
 [   1461     182]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.03      0.11      0.05      1643

    accuracy                           0.99   1272524
   macro avg       0.51      0.55      0.52   1272524
weighted avg       1.00      0.99      1.00   1272524

ROC-AUC: 0.5529621856200594


In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns

X_normal = X_scaled[y == 0]

input_dim = X_normal.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
from tensorflow.keras.losses import MeanSquaredError
autoencoder.compile(optimizer=Adam(0.001), loss=MeanSquaredError())



autoencoder.fit(X_normal, X_normal,
                epochs=5,
                batch_size=512,
                shuffle=True,
                validation_split=0.2)

X_pred = autoencoder.predict(X_scaled)
reconstruction_error = np.mean(np.square(X_scaled - X_pred), axis=1)

threshold = np.percentile(reconstruction_error, 90)  
y_pred = (reconstruction_error > threshold).astype(int)


print("Autoencoder Evaluation:")
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))
import joblib
autoencoder.save('autoencoder_model.keras')



Epoch 1/5
9929/9929 ━━━━━━━━━━━━━━━━━━━━ 13s 982us/step - loss: 0.0357 - val_loss: 0.0284
Epoch 2/5
9929/9929 ━━━━━━━━━━━━━━━━━━━━ 10s 948us/step - loss: 0.0016 - val_loss: 0.0225
Epoch 3/5
9929/9929 ━━━━━━━━━━━━━━━━━━━━ 10s 948us/step - loss: 0.0016 - val_loss: 0.0286
Epoch 4/5
9929/9929 ━━━━━━━━━━━━━━━━━━━━ 10s 952us/step - loss: 8.1545e-04 - val_loss: 0.0351
Epoch 5/5
9929/9929 ━━━━━━━━━━━━━━━━━━━━ 10s 988us/step - loss: 0.0010 - val_loss: 0.0387
198832/198832 ━━━━━━━━━━━━━━━━━━━━ 66s 331us/step
Autoencoder Evaluation:
[[5724435  629972]
 [   1923    6290]]
              precision    recall  f1-score   support

           0       1.00      0.90      0.95   6354407
           1       0.01      0.77      0.02      8213

    accuracy                           0.90   6362620
   macro avg       0.50      0.83      0.48   6362620
weighted avg       1.00      0.90      0.95   6362620



In [34]:
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,amount_normalized,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest,timestamp
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,-0.281560,False,False,True,False,0.0,9839.64,2025-01-01 00:00:00
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,-0.294767,False,False,True,False,0.0,1864.28,2025-01-01 00:01:00
2,1,181.00,181.0,0.00,0.0,0.0,1,0,-0.297555,False,False,False,True,0.0,181.00,2025-01-01 00:02:00
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,-0.297555,True,False,False,False,0.0,21363.00,2025-01-01 00:03:00
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,-0.278532,False,False,True,False,0.0,11668.14,2025-01-01 00:04:00


In [40]:
import smtplib
from email.mime.text import MIMEText
import pandas as pd

def send_alert(transaction):
    print("🚨 Fraud Detected in Transaction")
    
    sender_email = "deepb2601@gmail.com"
    receiver_email = "deepu26jan2004@gmail.com"
    password = "efww jpkb rwry valb"  # Make sure this is an app password

    subject = "🚨 Fraud Alert Detected"
    body = f"""
    Fraudulent transaction detected!

    Details:
    {transaction.to_string(index=True)}
    """

    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("✅ Email alert sent successfully!")
    except Exception as e:
        print(f"❌ Failed to send alert: {e}")


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   step               int64         
 1   amount             float64       
 2   oldbalanceOrg      float64       
 3   newbalanceOrig     float64       
 4   oldbalanceDest     float64       
 5   newbalanceDest     float64       
 6   isFraud            int64         
 7   isFlaggedFraud     int64         
 8   amount_normalized  float64       
 9   type_CASH_OUT      bool          
 10  type_DEBIT         bool          
 11  type_PAYMENT       bool          
 12  type_TRANSFER      bool          
 13  errorBalanceOrig   float64       
 14  errorBalanceDest   float64       
 15  timestamp          datetime64[ns]
dtypes: bool(4), datetime64[ns](1), float64(8), int64(3)
memory usage: 606.8 MB


In [41]:
sample_transaction = pd.Series({
    'amount': 4500,
    'type_TRANSFER': 1,
    'oldbalanceOrg': 5000,
    'newbalanceOrig': 500,
    'oldbalanceDest': 0,
    'newbalanceDest': 4500
})

send_alert(sample_transaction)


🚨 Fraud Detected in Transaction
✅ Email alert sent successfully!
